In [1]:
#!pip install xmltodict

In [2]:
import xmltodict
import json

In [3]:
with open("data/TheNewYorkTimes.1980.xml") as xml_file:
    data_dict = xmltodict.parse(xml_file.read())

In [4]:
len(data_dict['records']['record'])

94697

In [5]:
json_data = json.dumps(data_dict)